# Data Analysis with Pandas: split-apply-combine

Last week, we learned:
- Pandas is a library in Python that is designed for data manipulation and analysis
- How to use libraries (import them, access their functions and data structures with `library.function_name()`)
- About the `dataframe` data structure: basically a smart spreadsheet, with rows of observations, and columns of variables/data for each observation - sort of a cross between a list (sortable, indexable) and a dictionary (quickly access data by key)
- Some basic operations: constructing a dataframe, summarizing, subsetting, reshaping

This week, we'll learn a bit more about summarization:
- Use `.value_counts()` to summarize categorical data
- Use `.crosstab()` to summarize categorical data cross multiple columns

And more advanced operations for reshaping/modifying your dataframe:
- Use `.apply()` to apply functions to one or more columns to generate new columns
- Use `.groupby()` to split your data into subgroups, apply some function to their data, then combine them into a new dataframe for further analysis (the "**split-apply-combine**" pattern that is fundamental to data analysis with pandas)
- Use some basic plotting functions to explore your data

I'll then tie it all together to show how they map to problem formulations for your Project 4: all the projects have the same basic structure! These roughly correspond to Qs 6-8 in your PCEs.

# Setup

The files we will be working with in this session are the datasets used for Project 4:

<ul>
    <li><button data-commandlinker-command="docmanager:open" data-commandlinker-args='{"path": "/repos/INST-126-intro-to-programming-for-infosci/notebooks/data/bls-by-category.csv", "text": "testudo_fall2020.csv"}'>View bls-by-category.csv</button>
    </li>
    <li><button data-commandlinker-command="docmanager:open" data-commandlinker-args='{"path": "/repos/INST-126-intro-to-programming-for-infosci/notebooks/data/BreadBasket_DMS.csv", "text": "testudo_fall2020.csv"}'>View BreadBasket_DMS.csv</button></li>
    <li><button data-commandlinker-command="docmanager:open" data-commandlinker-args='{"path": "/repos/INST-126-intro-to-programming-for-infosci/notebooks/data/ncaa-team-data.csv", "text": "testudo_fall2020.csv"}'>View ncaa-team-data.csv</button></li>
    <li><button data-commandlinker-command="docmanager:open" data-commandlinker-args='{"path": "/repos/INST-126-intro-to-programming-for-infosci/notebooks/data/testudo_fall2020.csv", "text": "testudo_fall2020.csv"}'>View testudo_fall2020.csv</button> (for this one, pass <code>keep_default_na=False</code> when using <code>read_csv</code>)</li>
</ul>

In [ ]:
import pandas as pd

fn = 'data/testudo_fall2020.csv' # Note: on Windows use data\testudo_fall2020.csv instead!

# read in the file into a dataframe called courses
courses = pd.read_csv(fn, keep_default_na=False)

# use the .head() function to show the top 5 rows in the dataframe
courses.head(5) 

Here is a quick summary of the dataframe, rounded to two decimal digits. Since `.describe()` works only on numerical columns, the only entry showed is the `credits` column.

In [ ]:
courses.describe().round(2) 

## Use `.value_counts()` to summarize categorical data in your dataframe

Another way to get a summary of one or more columns that are *categorical*. The counts correspond to how many time each particular _category_ (a value) appears in the column. Results are sorted in descending order by default.

In [ ]:
# access the area column in the courses dataframe
area = courses['area']

# and apply the value_counts method to that column, which is a series data structure
area.value_counts()

Note that the result series is labeled with the possible values of the `area` column. (This is the major of the course.) As we saw last week, a Pandas series is like a cross between a dictionary and a list. So we can now use this new series to retrieve the counts for each individual area/major.

In [ ]:
# same as above but stored in a new variable
area_counts = courses['area'].value_counts()

# can get value by named key like a dict
area_code = "INST"
print(f"Count for {area_code=}: {area_counts[area_code]}")

<br/><br/><br/>

We can also look up counts by integer position:

In [ ]:
# The first entry in the series
print(f"The first major has {area_counts.iloc[0]} courses")

<br/>
<br/>
<br/>

However we don't know what major is that! To know what is the label associated to a particular integer position, we can make use of the `.index` attribute of a Pandas Series/Dataframe:

In [ ]:
# and also by location
lab0 = area_counts.index[0]
val0 = area_counts.iloc[0]
print(f"The first major is {lab0} and has {val0} courses")

<br/>
<br/>
<br/>

Note that `.value_counts()` returns results sorted in descending order, so the first entry is also the maximum. So the above could be achieved used the `.max()` / `.idxmax()` methods of pandas Series/Dataframe objects:

In [ ]:
print(f"The major with most courses is {area_counts.idxmax()} with {area_counts.max()} courses")

<br/><br/><br/> To list all the keys use the `.keys()` method:

In [ ]:
area_counts.keys()

<br/><br/><br/>This is the same as the `.index`:

In [ ]:
area_counts.keys() is area_counts.index

<br/><br/><br/>Let's say we want the top 5 most populous areas. We can slice/subset the series just like a list, and then get the keys from that subset.


In [ ]:
# use a slice (like a list) and then get keys (like a dict)
area_counts[:5].keys()

<br/><br/><br/><br/><br/>

---

## Coding Challenge \#1

How do we get the frequency counts for the items in the "bread basket" data frame?

In [ ]:
# Your code here
...

<br/><br/><br/><br/><br/><br/><br/><br/>

---

## Use `crosstabs()` to summarize categorical data across multiple columns 

If we have _multiple_ categorical columns, we may want to get the frequency of a particular combination of values from both columns.

Let's use the NCAA dataset to show this:

In [ ]:
# read data from CSV
fn = "data/ncaa-team-data.csv"  # Note: on Windows use data\ncaa-team-data.csv 
ncaa = pd.read_csv(fn, na_values=['NA'], keep_default_na=False)

# summarize first 5 rows
ncaa.head(5)

Let's explore the `ncaa_result` column.

In [ ]:
# All possible results
ncaa['ncaa_result'].value_counts()

Let's explore the `school` column

In [ ]:
ncaa['school'].value_counts()

What if I want to compute the counts of all possible combination of a school &times; a result? I could try to get the `.value_counts` of a sub-dataframe with these two columns only:

In [ ]:
ncaa[['school', 'ncaa_result']].value_counts()

<br/><br/><br/>

This however gives me a series, not a dataframe. The `crosstab` function instead organizes the same data in a tabular format (so as a dataframe instead of a series). It takes two pandas Series as parameters, which are going to correspond to two components of the cross-tabulaation:
1. The `index` parameter -- whose values are going to be the _rows_ of the cross-tab dataframe;
2. The `column` parameter -- whose values are going to be the _columns_ of the cross-tab dataframe.

In [ ]:
# Label rows by the school, and columns by the season result 
pd.crosstab(ncaa['school'], ncaa['ncaa_result'])

<br/><br/><br/>
Note that in the dataframe created by `pd.crosstab()` rows and columns are sorted in alphabetical order. While this may be OK for schools, it is definitely not the most intuitive for the various possible outcomes in the various NCAA tournaments.

Can we reorganize the columns of a dataframe? There is a neat trick to do so, which relies on column _subsetting_ &mdash; the technique we saw last week that, given a dataframe, uses list indexing _on the columns_ to generate a new sub-dataframe having only a subset of the columns of the original dataframe. It turns out that the list indexing syntax _preserves the order of the columns_ in the sub-dataframe. So we can exploit this behavior to create a new &ldquo;sub&rdquo;-dataframe that has the _full_ set of columns, but in the order we want.

Here is an example:

In [ ]:
# The full set of columns -- sorted from most important to least important result
NCAA_RESULTS_SORTED = [
    "Won National Final",
    "Lost National Final",
    "Lost National Semifinal",
    "Lost Regional Final (Final Four)",
    "Lost Regional Final",
    "Lost Regional Semifinal",
    "Lost Third Round",
    "Lost Second Round",
    "Lost First Round",
    "Playing First Round",
    "Lost First Four",
    "Playing First Four",
    "Lost Opening Round"
]

# Label rows by the school, and columns by the season result, now sorted by descending outcome importance 
pd.crosstab(ncaa['school'], ncaa['ncaa_result'])[NCAA_RESULTS_SORTED]

<br/><br/><br/>

---

## Cleaning data for cross-tabulation with `.apply()`

Now that we know how cross-tabulation works, let's explore another column --- the `coaches` column:

In [ ]:
# All possible results
ncaa['coaches'].value_counts()

<br/><br/><br/>Now we would like to count how many times each coach had a particular result (column `ncaa_result`). One problem is that coaches can appear with different seasons (if they coached at different schools). Since we are interested in the performance of individual coaches, straight cross-tabulating the `coaches` column with `ncaa_result` would not work. 

So we need to _clean_ the `coaches` column to extract only the name without the season part. We can do this with a method of Pandas series/dataframes called `.apply()`. This requires us to define a &ldquo;cleaning&rdquo; function.

Cleaning data is a laborious activity. It requires knowing all the various idiosyncracies of how the data were created. Click on the button below and scroll horizontally to the `coaches` column, and inspect a few of the entries. 

<button data-commandlinker-command="docmanager:open" data-commandlinker-args='{"path": "/repos/INST-126-intro-to-programming-for-infosci/notebooks/23-Pandas-Split-Apply-Combine-Review/ncaa-team-data.csv", "text": "testudo_fall2020.csv"}'>View ncaa-team-data.csv</button>

If you looked through the data, you may have noticed a few quirks about how the information about coaches was recorded  in the NCAA data:
1. Most entries follow the format: `<first_name> <last_name> (<year>-<year>)`;
2. Some entries, however, include also a middle name: `<first_name> <middle_name> <last_name> (<year>-<year>)`;
3. Some entries have multiple coaches, separated by comma: `<first_name> <last_name> (<year>-<year>), <first_name> <last_name> (<year>-<year>)`; for simplicity, in these cases we want to consider only the first coach;
4. Finally, not all entries have a coach. Entries where the coach information is missing are recorded either as `"Unknown"`, `"No coach"`, or `"None"`; we want to keep those as is.

So knowing these specific facts about our dataset, here is a function that will clean the `coaches` column.

In [ ]:
def get_coach_name(x):
    """
    Extract the name of the (first) coach in the season
    """
    # Return missing / unknown coaches as is
    if x in ['Unknown', 'None', 'No coach']:
        return x
    
    # If multiple coaches are listed, keep only the first one
    if "," in x:
        x = x.split(",")[0]
        
    # split the string around spaces to isolate first, (middle,) last name, and years
    elements = x.split(' ')
    
    # take everything but the years (last element)
    coach_name = elements[:-1]
    
    # recombine together the first, (middle,), and last name and return it.
    coach_name = " ".join(coach_name)
    return coach_name

<br/><br/><br/>We can test that this function works as expected:

In [ ]:
# Should return "Bill Chandler"
get_coach_name("Bill Chandler (11-7)")

In [ ]:
# Should return "George Ireland" only
get_coach_name("George Ireland (8-6), Jerry Lyne (2-9)")

In [ ]:
# Should return "None"
get_coach_name("None")

In [ ]:
# Should return "No coach"
get_coach_name("No coach")

<br/><br/><br/>To apply this function to all the entries in the colum, we pass *the function* to the `.apply()` method. This method repeatedly calls your function passing all entries in the column one at a time as arguments.

In [ ]:
ncaa['coaches'].apply(get_coach_name)

<br/><br/><br/>We can now add this series as a cleaned column back into the dataframe:

In [ ]:
# apply get_coach_name function to each entry in column `coaches` and create new column `coach_name`
ncaa['coach_name'] = ncaa['coaches'].apply(get_coach_name)

# create a sub-dataframe with just two columns -- ncaa_result and coach_name
pd.crosstab(ncaa['coach_name'], ncaa['ncaa_result'])[NCAA_RESULTS_SORTED]

<br/><br/><br/>
One advantage of having the data in tabular format is that now we can apply all the techniques for subsetting/reshaping the data that we know!

For example, we can ask who are the people who won _some_ national finals but also lost _some_ regional finals. This uses subsetting with boolean indexing.

In [ ]:
# Store crosstab in a new variable
coach_results = pd.crosstab(ncaa['coach_name'], ncaa['ncaa_result'])[NCAA_RESULTS_SORTED]

# Use boolean indexing on two columns
coach_results[(coach_results["Won National Final"] > 0) & (coach_results['Lost Regional Final'] > 0)]

<br/><br/><br/>
It turns out that most of the people who won the national finals also lost a lot of regional finals, which tells us something about how difficult it is to win the nationals.

<br/><br/><br/>

---

## Coding Challenge \#2

Now do the same with the UMD courses data. Let's see how many areas offer introductory courses. Follow these steps:

1. To decide whether a course is an &ldquo;introductory&rdquo; one, define a function that checks if a title string contains the word `"Introduction"`.
2. Then apply the function to the `title` column and add the result as a new column in the dataframe.
3. Finally, cross-tabulate the area by this new column. Store the result in a variables called `intro_by_area`.

If all goes well this is the dataframe you should obtain:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>is_intro</th>
      <th>False</th>
      <th>True</th>
    </tr>
    <tr>
      <th>area</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>AMST</th>
      <td>6</td>
      <td>3</td>
    </tr>
    <tr>
      <th>BMGT</th>
      <td>51</td>
      <td>2</td>
    </tr>
    <tr>
      <th>CMSC</th>
      <td>36</td>
      <td>10</td>
    </tr>
    <tr>
      <th>COMM</th>
      <td>31</td>
      <td>0</td>
    </tr>
    <tr>
      <th>ECON</th>
      <td>63</td>
      <td>1</td>
    </tr>
    <tr>
      <th>ENSP</th>
      <td>4</td>
      <td>2</td>
    </tr>
    <tr>
      <th>ENTS</th>
      <td>4</td>
      <td>0</td>
    </tr>
    <tr>
      <th>INFM</th>
      <td>4</td>
      <td>1</td>
    </tr>
    <tr>
      <th>INST</th>
      <td>40</td>
      <td>7</td>
    </tr>
    <tr>
      <th>MATH</th>
      <td>40</td>
      <td>9</td>
    </tr>
    <tr>
      <th>PHSC</th>
      <td>5</td>
      <td>0</td>
    </tr>
    <tr>
      <th>PLCY</th>
      <td>28</td>
      <td>0</td>
    </tr>
    <tr>
      <th>PSYC</th>
      <td>34</td>
      <td>4</td>
    </tr>
    <tr>
      <th>SPHL</th>
      <td>7</td>
      <td>0</td>
    </tr>
    <tr>
      <th>STAT</th>
      <td>11</td>
      <td>4</td>
    </tr>
    <tr>
      <th>URSP</th>
      <td>7</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

In [ ]:
# Your code here
...

<br/><br/><br/><br/><br/>

---

Notice that `.crosstab()` works with numerical data and booleans too. But the column names in the new data frame (the boolean values `True` and `False`) are not very meaningful. We can rename them with `.rename()`.

(Make sure, in the previous cell, that you have stored the result of the cross-tabulation in a variable called `intro_by_area`.)

In [ ]:
new_names = {
    True: "Yes",
    False: "No"
}
intro_by_area = intro_by_area.rename(columns=new_names)
intro_by_area

<br/><br/><br/>Now compute the fraction of intro courses per area. This is:

$$
\frac{\rm yes}{({\rm yes} + {\rm no})}
$$

In [ ]:
intro_by_area['frac_intro'] = intro_by_area['Yes'] / (intro_by_area['Yes'] + intro_by_area['No'])
intro_by_area

<br/><br/><br/>
Reset the index if you want the `area` information to be usable for analysis/plotting/etc.

In [ ]:
intro_by_area = intro_by_area.reset_index()
intro_by_area

<br/><br/><br/><br/><br/><br/>

---

## Computing data based on one or more columns using `.apply()`

All these examples involved modifying or creating new columns! In data analysis, we often want to do things to data in our columns for data preparation / cleaning. Sometimes there is missing data we want to re-code, or there is data we want to re-describe or re-classify for our analysis. We can do this with a combination of functions and the `apply()` method. It comes in two flavors:
- With a single column (i.e. on a Series)
- With multiple columns (i.e. on a DataFrame)

### `.apply()` with a single column

The `prereqs` column gives a string description of the prerequisites for the course

In [ ]:
courses['prereqs'].head()

<br/><br/><br/>Let's say we want to have a `prereqs` column that is sortable. For example:

    0 = No prereqs 
    1 = has prereqs

<br/><br/><br/>

#### Step 1: Define the function you want to apply

This cell defines the function and tests it on some sample inputs to check it works

In [ ]:
# Step 1: define the function you want to apply
def has_prereq(prereq_descr):
    """
    Determine whether a pre-requisite description includes the string "None"
    """
    if "None" in prereq_descr:
        return 0
    else:
        return 1

# this should yield 1
prereq = "BMGT301; or instructor permission" 
print(f"With {prereq=!r}: {has_prereq(prereq)=}")

# this should yield 0
prereq = "None"
print(f"With {prereq=!r}: {has_prereq(prereq)=}")

<br/><br/><br/>
#### Step 2: Apply the function to a column

We can create a new column called `has_prereqs` to save this information in the dataframe

In [ ]:
# Step 2: apply it to one or more columns

# This applies the has_prereq() function to every row in the prereqs column in the courses data frame
courses['has_prereqs'] = courses['prereqs'].apply(has_prereq) 
courses.head(10)

<br/><br/><br/>We can crosstab the new column with the previous `is_intro` column to see how many introductory courses have pre-requisites

In [ ]:
pd.crosstab(courses['is_intro'], courses['has_prereqs'])

<br/><br/><br/>Interestingly, some courses that are called `"Introduction ..."` do have pre-requisites. We can use boolean indexing (in the o to see what are these courses!

In [ ]:
courses[(courses['is_intro'] == True) & (courses['has_prereqs'] == 1)]

In [ ]:
def has_intro(descr):
    if "intro" in descr.lower():
        return 1
    else:
        return 0

courses['has_intro'] = courses['description'].apply(has_intro)
courses.head(5)

<br/><br/><br/><br/><br/><br/>

---

#### What's happening under the hood

As another example, let's say I want to know how many courses of each level (100-, 200-, 300-level, etc.) we have in each area. We don't have that data in the dataset; at least not explicitly. Fortunately we can make it with some simple programming that you already know how to do! The problem here is, given a code (i.e., data from one column), how do we "extract" the area?

In [ ]:
# Step 1: define the function
def extract_level(code):
    """
    Given a course code (e.g. INST126) extract the course level (100)
    
    Note that this function assumes course code starts with 4-letter area
    """
    return code[4] + '00'

c = "CMSC250"
extract_level(c)

<br/><br/><br/>Let's see how this works! The `.apply()` function generates a list that is the same length as the input column's number of rows, with a corresponding value for each input 

(in this case, we have 414 rows in the data frame)

In [ ]:
# Step 2: apply the function
courses['level'] = courses['code'].apply(extract_level)
courses.head(10)

<br/><br/><br/>This is equivalent to calling `extract_level` repeatedly in a for loop:

In [ ]:
tmp = []
for i in range(len(courses)):
    row = courses.loc[i]
    code = row['code']
    level = extract_level(code)
    print(f"{i}: code={code}, level={level}")
    tmp.append(level)
courses['level'] = tmp

<br/><br/><br/>Another example with the bread basket data frame. Let's extract the hour of the day from the `Time` column and save it in a column called `Hour`

In [ ]:
def extract_hour(time):
    return int(time.split(":")[0])

bread['Hour'] = bread['Time'].apply(extract_hour)
bread.sort_values(by="Hour")

<br/><br/><br/><br/><br/><br/>

---

#### Step 3: Save the resulting data from the `.apply()` into a new / existing column

What if we want to save the results so we can use it later? We can simply assign it to a column, new or existing. 

Remember, pandas prefers immutability in general (return a new object instead of modifying the object), and sometimes enforces it. 

With `.apply()`, it's enforced: you can't do it in place, you have to assign the returned series to a new variable if you want it to persist. 

<br/><br/><br/><br/><br/>

---

### `.apply()` with data from multiple columns

What if you want to have a way to filter the courses in terms of "easy entry points" (i.e., both introductory *and* has no prerequisites)? That might also be interesting to analyze by area to see how many departments offer these easy entry points into the department for students from other departments.

Core thing we need to know here is that our `.apply()` will now apply a function that has a **row** as input, not an element of a single column. That way, we can access data from any column in the row: in this case, data from the `is_intro` and `has_prereq` columns. We tell `.apply()` to do this with the `axis` parameter.  We need to pass `axis=1` when we call `.apply()` so it knows to pass a row into the function, not just a single column element. See here for more details: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html

In [ ]:
# is_entry_point function
def is_entry_point(row):
    """
    Determine whether a course is an "entry point" based on these two conditions:
    - It is an "intro" course (is_intro = 1)
    - It has no prerequisites (has_prere = 1)
    """
    if row['is_intro'] == 1 and row['has_prereqs'] == 0: 
        return 1
    else:
        return 0

In [ ]:
# this should yield 1
test_row = {
    'is_intro': 1,
    'has_prereqs': 0
}
print(f"With test_row = {test_row} is_entry_point(test_row) = {is_entry_point(test_row)}")
      
# this should yield 0
test_row = {
    'is_intro': 1,
    'has_prereqs': 1
}

print(f"With test_row = {test_row} is_entry_point(test_row) = {is_entry_point(test_row)}")

In [ ]:
# Step 2 apply the function (to the whole data frame!) and save the result

# need to specify axis=1 to apply it to every row
courses['is_entrypoint'] = courses.apply(is_entry_point, axis=1) 
courses.head()

<br/><br/><br/>What courses are entry points?

In [ ]:
courses[courses['is_entrypoint'] == 1]

In [ ]:
majors = pd.crosstab(courses['area'], courses['is_entrypoint'])
majors['frac_entry_points'] = majors[1] / (majors[0] + majors[1])
majors

<br/><br/><br/><br/><br/>

---

## The split-apply-combine pattern (with `.groupby()`)

We have seen we can &ldquo;reshape&rdquo; a dataframe in various ways: sorting, summarizing, cross-tabulation, etc. Going more deeply on this path of &ldquo;reshaping&rdquo;, we often __want to compute data based on subsets of the data, grouped by some column__. For example, we might want to see how many departments offer &ldquo;easy&rdquo; entry point courses. 

We can do this with the "split-apply-combine" pattern, which is implemented in the `.groupby()` function.

Basically, it goes like this:

1. **Split** the data into subgroups (e.g., split courses into department subgroups)

2. **Apply** some computation on each subgroup (e.g., find number of easy entry points for each department subgroup)

3. **Combine** subgroup-computation information into an overall new dataframe that has subgroups as entries

More info in this tutorial https://pandas.pydata.org/docs/getting_started/intro_tutorials/06_calculate_statistics.html

<br/><br/><br/><br/><br/>

---

### 1. Split 

We use the `.groupby()` method to split a dataframe into subgroups based on the values of a column

Let's split the courses dataframe by area and see how many courses are in each split.

In [ ]:
# The total number of rows in the data frame
print(f"There are {len(courses)} rows in the data frame")

In [ ]:
# Now we print the rows in each split
for area, area_courses in courses.groupby('area'):
    print(f"There are {len(area_courses)} rows in the data frame for area = {area}")

In [ ]:
# Show the split from the last iteration of the loop: it is a dataframe (notice the area column)
area_courses

<br/><br/><br/>

#### Split the manual way

We use indexing to find subgroups of rows with a given value, then we can apply some summarization statistics, like the average credits.

In [ ]:
# get all the unique area values
course_areas = courses['area'].value_counts().keys()

# iterate through each unique area value
for area in course_areas:
    # get the subset of the course data that is associated with this area
    area_courses = courses[courses['area'] == area]
    
    # print the number of rows
    print(f"There are {len(area_courses)} rows in the data frame for area = {area}")

In [ ]:
# Show the split from the last iteration of the loop: it is a dataframe (notice the area column)
area_courses

<br/><br/><br/>Notice also that these numbers correspond to the output of `courses['area'].value_counts()`

In [ ]:
courses['area'].value_counts()

<br/><br/><br/><br/><br/>

---

### Properties of `.groupby()`

In general `.groupby()` returns an object of class `DataFrameGroupBy` that performs the split. We use this object to split the courses df into subsets grouped by area. The data frame is first sorted by the column with the groups. The object also works as an _iterator_: we can iterate through the resulting collection of dataframe subsets where each step in the iteration allows us to grab:
1. the name of the subset, which is the shared value (in this case area)
2. the subset dataframe (here called `area_courses`)

In [ ]:
courses.groupby('area')

<br/><br/><br/>On top of supporting iteration, objects returned by `.groupby()` also give you a number of methods/attributes:

- `.groupby(...).get(KEY)` &ndash; (_method_) returns the group associated to KEY
- `.groupby(...).ngroups` &ndash; (_attribute_) the number of groups
- `.groupby(...).groups` &ndash; (_attribute_) a dictionary whose keys are group names and the values the list of corresponding row indices. 

Here `...` represents the column with the groups.

In [ ]:
grouped = courses.groupby('area')

grouped.get_group('INST')

In [ ]:
grouped.get_group

In [ ]:
grouped.ngroups

In [ ]:
grouped.groups['INST']

<br/><br/><br/>So yet another way to split the dataframe in groups is to use the `.groups` dict

In [ ]:
grouped = courses.groupby('area')
for area in grouped.groups:
    idx = grouped.groups[area]  # <--- list of indices of the rows in this area
    area_df = courses.loc[idx]  # <--- need to pass list of indices to the .loc[] indexer
    print(f"There are {len(area_courses)} rows in the data frame for area = {area}")

<br/><br/><br/><br/><br/>

---

### 2. Apply and Combine

The "manual" way: apply and combine into a new dataframe we construct from scratch. First, let's recreate the three columns we built last time using `.apply()`

In [ ]:
def isintro(title):
    return 'Introduction' in title

courses['is_intro'] = courses['title'].apply(isintro)


def hasprereqs(description):
    return 'None' in description

courses['has_prereqs'] = courses['prereqs'].apply(hasprereqs)


def isentrypoint(row):
    if row['is_intro'] == 1 & row['has_prereqs'] == 0:
        return 1
    else:
        return 0

courses['is_entrypoint'] = courses.apply(isentrypoint, axis=1)

In [ ]:
# create an empty list to hold the new rows of the new COMBINED dataframe
tmp = []

# SPLIT the dataframe by area, and iterate through each split
for area, areaData in courses.groupby('area'): 
  
    # APPLY operations on the dataframe split
    # ---------------------------------------
    
    # count the number of entry point courses in the subarea
    num_entrypoints = areaData['is_entrypoint'].sum()
    
    # count the number of total courses in the subarea
    num_classes = len(areaData)
    

    # COMBINE the resulting subcomputation into a new dataset
    # -------------------------------------------------------
    entry = {
      'area': area, # each row is an area
      'num_entrypoints': num_entrypoints, 
      'num_classes': num_classes,
    }
    tmp.append(entry) 
    
# convert the list of new entries into a dataframe
entry_courses_by_area = pd.DataFrame(tmp)
entry_courses_by_area

<br/><br/><br/>Another example: what is busiest hour of the day at the restaurant?

In [ ]:
bread

<br/><br/><br/>First, we use `.apply()` again to extract the hour of the transaction from the `Time` column

In [ ]:
def gethour(time):
    return int(time.split(":")[0])

bread['hour'] = bread['Time'].apply(gethour)
bread

<br/><br/><br/>The use split-apply-combine pattern again

In [ ]:
# create an empty list to hold the new rows of the new COMBINED dataframe
tmp = []

# SPLIT the dataframe by area, and iterate through each split
for hour, hour_df in bread.groupby('hour'):
  
    # APPLY operations on the dataframe split
    # ---------------------------------------
    
    # count the number of entry point courses in the subarea
    num_transactions = len(hour_df)    

    # COMBINE the resulting subcomputation into a new dataset
    # -------------------------------------------------------
    entry = {
      'hour': hour, # each row is an hour
      'num_transactions': num_transactions, 
    }
    tmp.append(entry) 
    
# convert the list of new entries into a dataframe
transactions_by_hour = pd.DataFrame(tmp)
transactions_by_hour    

<br/><br/><br/><br/><br/>

---

#### Named aggregation: Shortcut apply-combine with `.groupby()` + `.agg()`

To make `.groupby()` more powerful, we tack on the `.agg()` function to it to tell pandas to *aggregate* particular columns in particular ways (e.g., count the number of entry point courses in a given department, vs. give an average *proportion* of classes that are entry points). This is also called [named aggregation](https://pandas.pydata.org/docs/user_guide/groupby.html#named-aggregation) in the pandas official documentation.

In [ ]:
# SPLIT by area
courses.groupby("area", as_index=False).agg(
    # APPLY these computations and COMBINE into a new data frame using .agg
    # ----------------------------------------------------------------------

    # 1: apply `.sum()` to the `is_entrypoint` column of each subgroup
    num_entrypoints=('is_entrypoint', "sum"), 
    # 2: apply `.count() to the `area` column of each subgroup (similar to taking len() of subgroup)
    num_classes=('area', "count")
)

<br/><br/><br/>Same with restaurant transactions data frame

In [ ]:
bread.groupby("hour", as_index=False).agg(
    num_transactions=("Time", "count")
)

<br/><br/><br/>Named aggregations is a relatively newer feature in Pandas. The old school method is much more convoluted. You may still see if sometimes on StackOverflow or other websites.

In [ ]:
bread.groupby("hour", as_index=False)[['Time']].count().rename(columns={'Time': 'num_transactions'})

<br/><br/><br/><br/><br/>

---

## Anatomy of combining `.groupby()` with `.agg()` via named aggregation

<img src="https://terpconnect.umd.edu/~gciampag/INST126/images/groupby.png" width="100%" />

<br/><br/><br/><br/><br/>

---

### Using groupby for further analysis

Sometimes we want to take the result of the split-apply-combined data frame and do further analysis on it. Recall we said an entry point course (introduction + no prereqs) is an &ldquo;easy&rdquo; to get a sense of what an area is like.  Areas with more entry points are more &ldquo;open&rdquo; to people wishing to change major for example. What areas are more open? Now that we now how may entry points courses are there per area, and also how many courses in total there are, we could compute an &ldquo;openness&rdquo; score.

Here is the aggregated data frame we built before:

In [ ]:
entry_courses_by_area = courses.groupby("area", as_index=False).agg(
    num_entrypoints=('is_entrypoint', "sum"), 
    num_classes=('area', "count")
)
entry_courses_by_area

<br/><br/><br/>Let's now compute the proportion of entry point classes, as a proxy for &ldquo;openness&rdquo;, and finally let's sort by that score.

In [ ]:
# step 1: define the function
def openness(row):
    return row['num_entrypoints'] / row['num_classes']

# step 2: apply the function and save the results
entry_courses_by_area['openness'] = entry_courses_by_area.apply(openness, axis=1)

# step 3: sort by openness and reset index (passing ignore_index=True argument)
entry_courses_by_area.sort_values(by="openness", ascending=False, ignore_index=True)

<br/><br/><br/><br/><br/>

---

### What can you aggregate?

<table class="colwidths-given table">
<colgroup>
<col style="width: 20%">
<col style="width: 80%">
</colgroup>
<thead>
<tr class="row-odd"><th class="head"><p>Function</p></th>
<th class="head"><p>Description</p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p><code class="xref py py-meth docutils literal notranslate"><span class="pre">mean()</span></code></p></td>
<td><p>Compute mean of groups</p></td>
</tr>
<tr class="row-odd"><td><p><code class="xref py py-meth docutils literal notranslate"><span class="pre">sum()</span></code></p></td>
<td><p>Compute sum of group values</p></td>
</tr>
<tr class="row-even"><td><p><code class="xref py py-meth docutils literal notranslate"><span class="pre">size()</span></code></p></td>
<td><p>Compute group sizes</p></td>
</tr>
<tr class="row-odd"><td><p><code class="xref py py-meth docutils literal notranslate"><span class="pre">count()</span></code></p></td>
<td><p>Compute count of group</p></td>
</tr>
<tr class="row-even"><td><p><code class="xref py py-meth docutils literal notranslate"><span class="pre">std()</span></code></p></td>
<td><p>Standard deviation of groups</p></td>
</tr>
<tr class="row-odd"><td><p><code class="xref py py-meth docutils literal notranslate"><span class="pre">var()</span></code></p></td>
<td><p>Compute variance of groups</p></td>
</tr>
<tr class="row-even"><td><p><code class="xref py py-meth docutils literal notranslate"><span class="pre">sem()</span></code></p></td>
<td><p>Standard error of the mean of groups</p></td>
</tr>
<tr class="row-odd"><td><p><code class="xref py py-meth docutils literal notranslate"><span class="pre">describe()</span></code></p></td>
<td><p>Generates descriptive statistics</p></td>
</tr>
<tr class="row-even"><td><p><code class="xref py py-meth docutils literal notranslate"><span class="pre">first()</span></code></p></td>
<td><p>Compute first of group values</p></td>
</tr>
<tr class="row-odd"><td><p><code class="xref py py-meth docutils literal notranslate"><span class="pre">last()</span></code></p></td>
<td><p>Compute last of group values</p></td>
</tr>
<tr class="row-even"><td><p><code class="xref py py-meth docutils literal notranslate"><span class="pre">nth()</span></code></p></td>
<td><p>Take nth value, or a subset if n is a list</p></td>
</tr>
<tr class="row-odd"><td><p><code class="xref py py-meth docutils literal notranslate"><span class="pre">min()</span></code></p></td>
<td><p>Compute min of group values</p></td>
</tr>
<tr class="row-even"><td><p><code class="xref py py-meth docutils literal notranslate"><span class="pre">max()</span></code></p></td>
<td><p>Compute max of group values</p></td>
</tr>
</tbody>
</table>

In [ ]:
def getname(coaches):
    try:
        return " ".join(coaches.split(" ")[:2])
    except AttributeError:
        return "No Coach"

ncaa['coach'] = ncaa['coaches'].apply(getname)

ncaa.groupby('coach', as_index=False).agg(
    best_wl=('wl', 'max')
)

<br/><br/><br/><br/><br/>

---

# Putting it all together

<img src="https://terpconnect.umd.edu/~gciampag/INST126/images/putting-together.png" width="100%" />

<br/><br/><br/><br/><br/>

---

## Reminder: More resources

* The pandas website is decent place to start: https://pandas.pydata.org/
* This "cheat sheet" is also a really helpful guide to more common operations that you may run into later: https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf
* There are also many blogs that are helpful, like towardsdatascience.com
* The cool thing about pandas and data analysis in python is that many people share notebooks that you can inspect / learn from / adapt code for your own projects (just like mine!).

<br/><br/><br/><br/><br/>

---

## EXTRA: Plotting

The main library for plotting in Python is `matplotlib`. You can learn that library later. It has lots of fine-grained controls. For now, you can use pandas "wrapper" over matplotlib (basically calling matplotlib from inside pandas), which is a bit easier to learn.

In [ ]:
# plot openness by area
entry_courses_by_area.sort_values(by='openness', ascending=False).plot(
    x="area", 
    y="openness", 
    kind='bar', 
    xlabel="Major", 
    ylabel="Proportion of entry point classes",
    title="Average openness by major"
)

In [ ]:
entry_courses_by_area.sort_values(by="num_classes", ascending=False).plot(
    x="area", 
    y="num_classes", 
    kind="bar",
    xlabel="Major",
    ylabel="Number of classes",
    title="Number of courses by major"
)

<br/><br/><br/><br/>
<br/><br/><br/><br/>
<br/><br/><br/><br/>
<br/><br/><br/><br/>
<br/><br/><br/><br/>
<br/><br/><br/><br/>
<br/><br/><br/><br/>
<br/><br/><br/><br/>
<br/><br/><br/><br/>
<br/><br/><br/><br/>
<br/><br/><br/><br/>
<br/><br/><br/><br/>

---

# Solutions

## Coding Challenge \#1

In [ ]:
### BEGIN SOLUTION
bread = pd.read_csv('data/BreadBasket_DMS.csv')  # Note: on Windows use data\BreadBasket_DMS.csv
bread['Item'].value_counts()
### END SOLUTION

## Coding Challenge \#2

In [ ]:
### BEGIN SOLUTION
def is_intro(title):
    """ 
    Determine whether a course title has the word "Introduction" or not in it
    """
    if "Introduction" in title:
        return True
    else:
        return False
        
# apply the is_intro function to the title column in courses
# and save the results in the is_intro column in courses
courses['is_intro'] = courses['title'].apply(is_intro)

# cross-tabulate area and is_intro
intro_by_area = pd.crosstab(courses["area"], courses["is_intro"])
intro_by_area
### END SOLUTION